In [ ]:
import logging

from IPython.core.display import display, HTML

from scripts.main import *
from scripts.retrieve_data import *
from scripts.extract_data import*

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

###  Read in all the data files day by day and search for problems

In [ ]:
config = load_config()

# get the date range from the config
start_date = dt.strptime(config['main']['start_date'], '%Y-%m-%d')
end_date = dt.strptime(config['main']['end_date'], '%Y-%m-%d')
days_range = pd.date_range(start_date, end_date)

# create the variable holding all the series for all days
df_series = None

# go through the date range day by day
for day in days_range:

    config['main']['start_date'] = day.strftime('%Y-%m-%d')
    config['main']['end_date'] = day.strftime('%Y-%m-%d')
    
    df = extract_transform_and_save_data_from_files(config)
    if df is None: continue
    df_problem = df[
            (df['Start Time'].isnull())
            | (df['End Time'].isnull())
            | (df['Machine'] == '')
            | (df['Machine'].isnull())
            | (df['Institution Name'] == '')
            | (df['Institution Name'].isnull())]
    if len(df_problem) <= 0: continue
    logging.info('Current day: {}'.format(day.strftime('%Y-%m-%d')))
    with pd.option_context("display.max_colwidth", 1000): display(df_problem)
    with pd.option_context("display.max_colwidth", 1000): display(df[df['Series Instance UID'].isin(df_problem['Series Instance UID'])])